In [ ]:
%pip install jinja2

In [ ]:
# The output from Terraform contains information about Azure resources
# and about the projects that the clients have.
import subprocess
def get_terraform_output():
    result = subprocess.run(
        ['terraform', 'output', '-json'],
        capture_output=True,
        text=True,
        cwd="../iac-terraform/"
    )
    if result.returncode != 0:
        print(f"Error running terraform output: {result.stderr}")
        return None
    else:
        return result.stdout

output = get_terraform_output()
assert output is not None
assert output.strip() != ""

In [ ]:
# Extract the information from the format Terraform uses for outputs
import json
from pprint import pprint
output_json = json.loads(output)
envs = {
    env: output_json["envs"]["value"][env]
    for env in output_json["envs"]["value"].keys()
}
print("Found environments to be used in templating!")
pprint(list(envs.keys()))
for k,v in envs["dev_1"].items():
    print("\t",k,type(v))
projects = {
    app: output_json["projects"]["value"][app]
    for app in output_json["projects"]["value"].keys()
}
print("Found projects to be used in templating!")
projects = {
    app: output_json["projects"]["value"][app]
    for app in output_json["projects"]["value"].keys()
}
pprint(list(projects.keys()))
for k,v in projects["cluster-stuff"].items():
    print("\t",k,type(v))


In [ ]:
# Discover workload folders which may contain templates
from glob import glob
from pathlib import Path
workloads = [Path(x) for x in glob("./workloads/*")]

assert len(workloads)<10 # just in case lol
print("Found workloads!")
pprint(workloads)

In [ ]:
# Preview a project
print(json.dumps(projects["self-service-portal"], indent=4))

In [ ]:
def get_work(
    workload_dir: Path,
    templates_dir,
    template_path: Path,
    envs: dict,
    projects: dict
):
    current = [{
        "template_file": str(template_path),
        "destination_path": str(workload_dir / template_path.relative_to(templates_dir)),
        "variables": {
            "envs": envs,
            "projects": projects,
        }
    }]
    new = []
    for work in current:
        if "{{env}}" in work["destination_path"]:
            for env_name, env_data in envs.items():
                new.append({
                    **work,
                    "destination_path": work["destination_path"].replace("{{env}}", env_name),
                    "variables": {
                        **work["variables"],
                        "env": {**env_data, "name": env_name},
                    },
                })
        else:
            new.append(work)
    current = new

    new = []
    for work in current:
        if "{{project}}" in work["destination_path"]:
            for proj_name, proj_data in projects.items():
                if "{{app}}" in work["destination_path"]:
                    for app_name, app_data in proj_data["apps"].items():
                        new.append({
                            **work,
                            "destination_path": work["destination_path"].replace("{{project}}", proj_name).replace("{{app}}", app_name),
                            "variables": {
                                **work["variables"],
                                "project": {**proj_data, "name": proj_name},
                                "app": {**app_data, "name": app_name},
                            },
                        })
                else:
                    new.append({
                        **work,
                        "destination_path": work["destination_path"].replace("{{project}}", proj_name),
                        "variables": {
                            **work["variables"],
                            "project": {**proj_data, "name": proj_name},
                        },
                    })
        else:
            new.append(work)
    current = new

    return current

In [ ]:
# Separate discovery from fulfillment
work = []
"""
work: {
    template_file: str
    destination_path: str
    variables: dict
}[]
"""

for workload_dir in workloads:
    templates_dir = workload_dir / "templates"
    if templates_dir.exists():
        print(f"Workload {workload_dir} uses templates")
        found_yamls = list(templates_dir.glob("**/*.yaml"))
    
        for yaml_path in found_yamls:
            print(yaml_path)
            work.extend(get_work(workload_dir, templates_dir, yaml_path, envs, projects))
        print()
    continue
print(f"After expansion, {len(work)} yamls to be created")

In [ ]:
from jinja2 import Environment, StrictUndefined, UndefinedError
warning_header = """
# THIS FILE IS MANAGED BY AUTOMATION!
# IF YOU WANT TO MAKE CHANGES, EDIT THE TEMPLATE AND RE-RUN THE AUTOMATION
# GO TO `iac-kubernetes/manifest_templates.ipynb` TO FIND THE AUTOMATION
""".strip()
jinja = Environment(undefined=StrictUndefined)
for x in work:
    assert x["destination_path"] != x["template_file"]
    assert "templates" not in str(x["destination_path"])
    content = open(x["template_file"], encoding="utf-8").read()
    if "$" in content:
        print("Found dollar sign in template, jinja doesn't use dollar signs, are you sure your template is correct?", x["template_file"])
    content = jinja.from_string(content)
    try:
        content = content.render(x["variables"])
    except UndefinedError as e:
        print(f"Problem rendering template")
        import re
        print(re.sub(r'": ".*?"', '": "omitted"', json.dumps(x, indent=4)))
        raise e

    content = warning_header + "\n" + content
    dest = jinja.from_string(str(x["destination_path"]))
    dest = dest.render(x["variables"])
    Path(dest).parent.mkdir(exist_ok=True, parents=True)

    print(f"Rendering {dest}")
    with open(dest, "w") as f:
        f.write(content)

# Sanity check

we want to warn if a template is not references in the kustomization.yaml

In [ ]:
%pip install pyyaml

In [ ]:
import yaml
from pathlib import Path

def find_nearest_kustomization(path):
    """
    Find the nearest kustomization.yaml in the parent directories.
    """
    current_path = path.parent
    while current_path != Path("/"):
        kustomization = current_path / "kustomization.yaml"
        if kustomization.exists():
            return kustomization
        current_path = current_path.parent
    return None

warnings = []

for x in work:
    seek = Path(x["destination_path"])
    seek = seek.relative_to(list(seek.parents)[-5])
    if seek.name == "kustomization.yaml":
        continue

    kustomization = find_nearest_kustomization(Path(x["destination_path"]))
    
    if not kustomization:
        warnings.append(f"Could not find kustomization file near {x['destination_path']}")
    else:
        with open(kustomization, 'r') as file:
            data = yaml.safe_load(file)
            
            # check if seekName is referenced in patches or resources
            inPatches = "patches" in data and any(patch for patch in data["patches"] if patch["path"] == str(seek).replace("\\", "/"))
            inResources = "resources" in data and any(res for res in data["resources"] if res == str(seek).replace("\\", "/"))
            if not inPatches and not inResources:
                warnings.append(f"Kustomization at {kustomization} doesn't reference {seek}")

for project_name, project_data in projects.items():
    mine = set(app["k8s_namespace"] for app in project_data["apps"].values())
    others = set(app["k8s_namespace"] for other_project_name, other_project_data in projects.items() for app in other_project_data["apps"].values() if other_project_name != project_name)
    if len(mine.intersection(others)) > 0:
        warnings.append(f"Projects must not share namespaces! {project_name}:{mine} shared {mine.intersection(others)}")


print(json.dumps(warnings, indent=4))
